In [10]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [ ]:
# Clear any logs from previous runs
rm -rf ./logs/

## Initial setup

* Import ResNet50 pretrained model

In [12]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

batch_size = 32
img_height = 224
img_width = 224
img_size = (img_height, img_width)
img_shape = img_size + (3,)

pre_trained_model = ResNet50(input_shape = img_shape,
                        include_top = False,
                        weights = 'imagenet')

print("Number of layers: ", len(pre_trained_model.layers))

Number of layers:  175


* Load dataset

In [22]:
# Edit according to local path for dataset
ds_path = r"fruitveg"

train_ds = tf.keras.utils.image_dataset_from_directory(ds_path,
                                        validation_split = 0.2,
                                        subset = "training",
                                        seed = 123,
                                        image_size = img_size,
                                        batch_size = batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(ds_path,
                                      validation_split = 0.2,
                                      subset = "validation",
                                      seed = 123,
                                      image_size = img_size,
                                      batch_size = batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)

Found 4136 files belonging to 18 classes.
Using 3309 files for training.
Found 4136 files belonging to 18 classes.
Using 827 files for validation.


* Hyperparameter setup

In [20]:
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(1e-5, 1e-1))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.4))
HP_TUNING_LAYER = hp.HParam('tuning_layers', hp.IntInterval(150, 175))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_LEARNING_RATE, HP_DROPOUT, HP_TUNING_LAYER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

## Define training & run function

In [24]:
def model_training(hparams):
    pre_trained_model.trainable = True

    # Fine-tune from this layer onwards
    START_TRAIN = hparams[HP_TUNING_LAYER]

    # Freeze all the layers before 
    for layer in pre_trained_model.layers[:START_TRAIN]:
        layer.trainable = False

    # Averaging layer
    global_average = tf.keras.layers.GlobalAveragePooling2D()

    # Data augmentation 
    augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal_and_vertical'),
        tf.keras.layers.RandomRotation(0.2)
    ])

    # Add dense layer
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

    # Chain model 
    inputs = tf.keras.Input(shape = img_shape)
    x = augmentation(inputs) 
    x = preprocess_input(x)
    x = pre_trained_model(x, training=False)
    x = global_average(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs,outputs)
    
    # Compile model
    LR = hparams[HP_LEARNING_RATE]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                  metrics=['accuracy'])
    
    # Train model
    EPOCH = 5

    model.fit(train_ds,
        validation_data = val_ds,
        epochs = EPOCH,
        callbacks=[
            tf.keras.callbacks.TensorBoard(logdir),  
            hp.KerasCallback(logdir, hparams)
            ]
        )
    _, accuracy = model.evaluate(train_ds, val_ds)
    return accuracy

In [17]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = model_training(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

## Start runs

In [ ]:
session_num = 0
logdir = r'logs/hparam_tuning'

for learning_rate in (HP_LEARNING_RATE.domain.min_value,
                        HP_LEARNING_RATE.domain.max_value):
    for dropout_rate in (HP_DROPOUT.domain.min_value,
                         HP_DROPOUT.domain.max_value):
        for layer in (HP_TUNING_LAYER.domain.min_value,
                             HP_TUNING_LAYER.domain.max_value):
            hparams = {
                  HP_LEARNING_RATE: learning_rate,
                  HP_DROPOUT: dropout_rate,
                  HP_TUNING_LAYER: layer,
              }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'learning_rate': 1e-05, 'dropout': 0.1, 'tuning_layers': 150}
Epoch 1/5
 85/104 [=======================>......] - ETA: 1:05 - loss: 2.5237 - accuracy: 0.2279

## Visualize in TensorBoard

In [ ]:
%tensorboard --logdir logs/hparam_tuning